# **1 - New York Taxi Trips**
- Qual ***vendor*** realizou mais viagens e a quantidade de viagens feitas agrupando os dados por estações do ano(outono, inverno, primavera e verão).
- Qual ***vendor*** percorreu a maior distância e a soma das distâncias em cada estação do ano.
- Qual o tipo de pagamento mais utilizado em casa estação do ano independente do ***vendor***.

**Entrega:**

Um documento SQL que cria tabelas, carrega as informações do dataset no banco e uma query que retorna as informações listadas acima, sugerimos a utilização do [DB Fiddle](https://www.db-fiddle.com/), avaliação será feita com Postgres 9.6.

# Preparação do Ambiente

In [ ]:
# install postgresql
!wget --quiet -O - https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo apt-key add -
!echo "deb http://apt.postgresql.org/pub/repos/apt/ $(lsb_release -cs)-pgdg main" | sudo tee /etc/apt/sources.list.d/postgresql-pgdg.list > /dev/null
!sudo apt update -y -q
!sudo apt install postgresql-9.6 -q


In [ ]:
# Start server
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
!\set autocommit on

In [ ]:
# Setup client with magic
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

In [ ]:
%%sql
SELECT version();

In [ ]:
!pip install --upgrade --no-cache-dir gdown

In [ ]:
!gdown '1DOvZ-lUlRwyc8jStSSe4Ps0kncHSvhkT'
!gdown '1ilCYiB72T8UPerLiku1c6qdRh94vAUhK'
!gdown '1-UD_8gnTO1UwW-ZQYbW-2WXlwAK7wsl4'
!gdown '10eAuCp7pdUzmBj1SuN_zae3Vo59Wsrfy'
!ls -lt

In [ ]:
!rm -rf /tmp/datasets
! mkdir /tmp/datasets
!cp data-nyctaxi-trips-2009.json /tmp/datasets
!cp data-nyctaxi-trips-2010.json /tmp/datasets
!cp data-nyctaxi-trips-2011.json /tmp/datasets
!cp data-nyctaxi-trips-2012.json /tmp/datasets

!sudo chown -R postgres:postgres /tmp/datasets/*

!ls -lt /tmp/datasets/

---

# DDL

In [ ]:
%%sql
 SELECT current_database();

In [ ]:
%%sql 
DROP TABLE IF EXISTS temp;
CREATE TABLE temp (data JSONB);

In [ ]:
!psql -U root -d postgres -c "\COPY temp (data) FROM '/tmp/datasets/data-nyctaxi-trips-2009.json';"
!psql -U root -d postgres -c "\COPY temp (data) FROM '/tmp/datasets/data-nyctaxi-trips-2010.json';"
!psql -U root -d postgres -c "\COPY temp (data) FROM '/tmp/datasets/data-nyctaxi-trips-2011.json';"
!psql -U root -d postgres -c "\COPY temp (data) FROM '/tmp/datasets/data-nyctaxi-trips-2012.json';"

In [ ]:
# Validation
# 2.96 s ± 133 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# 16.7 s ± 1.92 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
%%sql
SELECT data->>'pickup_datetime'
FROM temp
ORDER BY data->>'pickup_datetime' ASC
LIMIT 1;

In [ ]:
# Validation
%%sql
SELECT data->>'pickup_datetime'
FROM temp
ORDER BY data->>'pickup_datetime' DESC
LIMIT 1;

### Inserindo o JSON data dentro de uma table

In [ ]:
%%sql
DROP TABLE IF EXISTS trips;

In [ ]:
%%sql
CREATE TABLE trips (
    id SERIAL,
    vendor_id VARCHAR(3),
    pickup_datetime TIMESTAMP WITH TIME ZONE,
    dropoff_datetime TIMESTAMP WITH TIME ZONE,
    passenger_count SMALLINT,
    trip_distance NUMERIC,
    pickup_longitude NUMERIC,
    pickup_latitude NUMERIC,
    rate_code SMALLINT,
    store_and_fwd_flag TEXT,
    dropoff_longitude NUMERIC,
    dropoff_latitude NUMERIC,
    payment_type VARCHAR(10),
    fare_amount NUMERIC,
    surcharge NUMERIC,
    tip_amount NUMERIC,
    tolls_amount NUMERIC,
    total_amount NUMERIC,
    CONSTRAINT PK_id PRIMARY KEY (id)
);

SELECT 
	column_name,
  data_type
FROM information_schema.columns
WHERE table_name = 'trips';

---

# DML

In [ ]:
# JSONB: 56.1 s ± 2.08 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
# JSON: 4min 59s ± 8.52 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
%%sql
INSERT INTO trips (
    vendor_id,
    pickup_datetime,
    dropoff_datetime,
    passenger_count,
    trip_distance,
    pickup_longitude,
    pickup_latitude,
    rate_code,
    store_and_fwd_flag,
    dropoff_longitude,
    dropoff_latitude,
    payment_type,
    fare_amount,
    surcharge,
    tip_amount,
    tolls_amount,
    total_amount
)
(
  SELECT 
    (data->> 'vendor_id')::varchar(3),
    (data->> 'pickup_datetime')::timestamp with time zone,
    (data->> 'dropoff_datetime')::timestamp with time zone,
    (data->> 'passenger_count')::smallint,
    (data->> 'trip_distance')::numeric,
    (data->> 'pickup_longitude')::numeric,
    (data->> 'pickup_latitude')::numeric,
    (data->> 'rate_code')::smallint,
    (data->> 'store_and_fwd_flag')::text,
    (data->> 'dropoff_longitude')::numeric,
    (data->> 'dropoff_latitude')::numeric,
    (data->> 'payment_type')::varchar(10),
    (data->> 'fare_amount')::numeric,
    (data->> 'surcharge')::numeric,
    (data->> 'tip_amount')::numeric,
    (data->> 'tolls_amount')::numeric,
    (data->> 'total_amount')::numeric
  FROM temp
);

In [ ]:
# Validation
%%sql
SELECT * FROM trips
LIMIT 2;

In [ ]:
%%sql
DROP TABLE temp;

## **Qual vendor realizou mais viagens e a quantidade de viagens feitas agrupando os dados por estações do ano(outono, inverno, primavera e verão)**

In [ ]:
# To get the seasons, I took the column pickup_datetime
%%sql
WITH seasons AS (
    SELECT
        CASE
            WHEN date_part('month', pickup_datetime) IN (12, 1, 2) THEN 'Verão'
            WHEN date_part('month', pickup_datetime) IN (3, 4, 5) THEN 'Outono'
            WHEN date_part('month', pickup_datetime) IN (6, 7, 8) THEN 'Inverno'
            WHEN date_part('month', pickup_datetime) IN (9, 10, 11) THEN 'Primavera'
        END AS estacao,
        vendor_id AS vendor,
        COUNT(*) as total_viagens
    FROM trips
    GROUP BY estacao, vendor_id
    ORDER BY estacao, total_viagens DESC
),
rank_vendors_by_seasons AS (
  SELECT
    estacao,
    vendor,
    total_viagens,
    DENSE_RANK() OVER (PARTITION BY estacao ORDER BY total_viagens DESC) as rank
  FROM seasons
)

SELECT 
    vendor,    
    total_viagens,
    estacao
FROM rank_vendors_by_seasons
WHERE rank = 1;

In [ ]:
# Validation
%%sql
SELECT
  vendor_id, 
  COUNT(vendor_id) AS total_viagens
FROM trips
GROUP BY vendor_id
ORDER BY total_viagens DESC

## **Qual vendor percorreu a maior distância e a soma das distâncias em cada estação do ano**

In [ ]:
# To get the distances traveled, I took the col col trip_distance
%%sql
WITH seasons AS (
    SELECT
        CASE
            WHEN date_part('month', pickup_datetime) IN (12, 1, 2) THEN 'Verão'
            WHEN date_part('month', pickup_datetime) IN (3, 4, 5) THEN 'Outono'
            WHEN date_part('month', pickup_datetime) IN (6, 7, 8) THEN 'Inverno'
            WHEN date_part('month', pickup_datetime) IN (9, 10, 11) THEN 'Primavera'
        END AS estacao,
        vendor_id AS vendor,
        SUM(trip_distance) as total_distancia
    FROM trips
    GROUP BY estacao, vendor_id
    ORDER BY estacao, total_distancia DESC
),
rank_trip_distance_by_seasons AS (
    SELECT 
      estacao,
      vendor,
      total_distancia,
      DENSE_RANK() OVER (PARTITION BY estacao ORDER BY total_distancia DESC) as rank
  FROM seasons
)
SELECT 
    vendor,
    total_distancia,
    estacao
FROM rank_trip_distance_by_seasons
WHERE rank = 1;

In [ ]:
# Validation
%%sql
SELECT 
  vendor_id, 
  SUM(trip_distance) as total_distancia
FROM trips
GROUP BY vendor_id
ORDER BY total_distancia DESC

## **Qual o tipo de pagamento mais utilizado em casa estação do ano independente do vendor**

In [ ]:
%%sql
WITH seasons AS (
    SELECT
        CASE
            WHEN date_part('month', pickup_datetime) IN (12, 1, 2) THEN 'Verão'
            WHEN date_part('month', pickup_datetime) IN (3, 4, 5) THEN 'Outono'
            WHEN date_part('month', pickup_datetime) IN (6, 7, 8) THEN 'Inverno'
            WHEN date_part('month', pickup_datetime) IN (9, 10, 11) THEN 'Primavera'
        END AS estacao,
        UPPER(payment_type) tipo_pagamento,
        COUNT(*) as total_pagamento
    FROM trips
    GROUP BY estacao, tipo_pagamento
    ORDER BY estacao, total_pagamento DESC
),
rank_payment_type_by_seasons AS (
  SELECT 
    estacao,
    tipo_pagamento,
    DENSE_RANK() OVER (PARTITION BY estacao ORDER BY total_pagamento DESC) as rank
  FROM seasons
)
SELECT 
    estacao,
    tipo_pagamento
FROM rank_payment_type_by_seasons
WHERE rank = 1;

In [ ]:
# Validatin
%%sql
SELECT 
  payment_type, 
  COUNT(*) as total_pagamento
FROM trips
GROUP BY payment_type
ORDER BY total_pagamento DESC

---